<a href="https://colab.research.google.com/github/BeckieHuang/WhetherRain/blob/main/M11015Q19_%E9%BB%83%E7%BF%94%E8%8E%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
# 載入相關套件
import pandas as pd
import numpy as np

from IPython.display import display
import warnings
warnings.filterwarnings( 'ignore' )

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [81]:
#　把資料表讀進來
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
submit = pd.read_csv('ex_submit.csv')

# 觀察資料表欄與列的數量、資料表內容
print( f'train {df_train.shape}' )
display( df_train.head() )

print( f'\ntest {df_test.shape}' )
display( df_test.head() )

train (17093, 23)


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Attribute21,Attribute22,Attribute23
0,2008-12-01,2,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-17,2,14.1,20.9,0.0,NaN,NaN,ENE,22.0,SSW,E,11.0,9.0,69.0,82.0,1012.2,1010.4,8.0,1.0,17.2,18.1,No,Yes
2,2008-12-20,2,9.8,25.6,0.0,NaN,NaN,SSE,26.0,SE,NNW,17.0,6.0,45.0,26.0,1019.2,1017.1,NaN,NaN,15.8,23.2,No,No
3,2008-12-30,2,12.5,24.2,1.2,NaN,NaN,WNW,50.0,WSW,SW,11.0,22.0,78.0,70.0,1005.6,1003.4,8.0,8.0,12.5,18.2,Yes,No
4,2009-01-22,2,24.4,34.0,0.6,NaN,NaN,NW,98.0,N,NNW,26.0,48.0,60.0,39.0,1005.3,997.8,4.0,1.0,26.1,33.1,No,Yes



test (790, 22)


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Attribute21,Attribute22
0,2009/1/2,10,18.4,28.9,0.0,14.8,13.0,S,37,SSE,SSE,19,19,30,8,1012.9,1012.1,1,1,20.3,27.0,No
1,2009/7/26,10,8.9,12.2,0.8,2.4,1.2,N,24,NNE,S,7,11,91,77,1018.1,1013.5,8,7,9.1,11.4,No
2,2010/1/19,10,11.3,28.8,0.0,11.2,12.9,SSW,50,S,SSW,9,20,29,12,1012.9,1011.2,0,0,18.1,26.5,No
3,2010/3/3,10,15.8,30.4,0.0,8.0,6.3,E,39,ESE,E,22,20,59,30,1013.7,1010.5,1,5,19.0,29.4,No
4,2010/6/25,10,10.0,22.0,0.0,3.0,7.7,NNW,39,NNE,NW,20,19,73,52,1022.3,1019.0,1,6,13.7,20.8,No


In [82]:
# 統計欄位缺失值總數的函數
def Missing_Counts( Data ) : 
    missing = Data.isnull().sum()  # 計算欄位中缺失值的數量 
    missing = missing[ missing>0 ]
    missing.sort_values( inplace=True ) 
    
    Missing_Count = pd.DataFrame( { 'ColumnName':missing.index, 'MissingCount':missing.values } )
    Missing_Count[ 'Percentage(%)' ] = Missing_Count['MissingCount'].apply( lambda x:round(x/Data.shape[0]*100,2) )
    return  Missing_Count

In [83]:
print( 'train :' )
display( Missing_Counts(df_train) )

print( 'test :' )
display( Missing_Counts(df_test) )

train :


,ColumnName,MissingCount,Percentage(%)
0,Attribute4,45,0.26
1,Attribute3,59,0.35
2,Attribute20,96,0.56
3,Attribute12,152,0.89
4,Attribute5,158,0.92
5,Attribute22,158,0.92
6,Attribute14,198,1.16
7,Attribute13,285,1.67
8,Attribute21,304,1.78
9,Attribute15,412,2.41


test :


,ColumnName,MissingCount,Percentage(%)


In [84]:
# 把訓練資料中的缺失值(null)丟掉
df_train.dropna(inplace=True)

In [85]:
# 把train與test資料表合併
df_data = df_train.append( df_test )
df_data

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Attribute21,Attribute22,Attribute23
706,2009-01-04,10,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
707,2009-01-07,10,27.1,36.1,0.0,13.0,0.0,N,43.0,N,WNW,7.0,20.0,26.0,19.0,1007.7,1007.4,8.0,8.0,30.7,34.3,No,No
708,2009-01-08,10,23.3,34.0,0.0,9.8,12.6,SSW,41.0,S,SSE,17.0,19.0,33.0,15.0,1011.3,1009.9,3.0,1.0,25.0,31.5,No,No
709,2009-01-18,10,17.9,34.2,0.0,12.0,11.3,SE,61.0,NE,SE,22.0,17.0,46.0,19.0,1016.4,1013.5,1.0,2.0,21.6,32.2,No,No
710,2009-01-21,10,25.4,33.5,0.0,13.6,3.7,N,46.0,NW,N,9.0,28.0,46.0,52.0,1012.0,1009.8,4.0,7.0,28.9,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,2016/4/22,13,24.9,35.9,0.0,6.0,11.0,E,41.0,E,SE,11.0,22.0,68.0,26.0,1012.6,1009.0,1.0,4.0,30.1,34.8,No,NaN
786,2016/4/30,13,25.2,34.2,0.0,4.4,8.0,ENE,31.0,S,NW,9.0,17.0,75.0,62.0,1012.8,1008.6,6.0,7.0,29.3,32.0,No,NaN
787,2016/11/6,13,22.7,31.4,25.6,7.0,3.4,SSE,41.0,NNW,NNW,7.0,11.0,79.0,93.0,1011.9,1008.5,7.0,7.0,30.4,25.9,Yes,NaN
788,2016/12/26,13,25.4,33.4,0.0,4.8,6.3,W,52.0,ENE,WNW,7.0,13.0,85.0,55.0,1006.6,1004.5,8.0,7.0,28.0,32.3,No,NaN


In [86]:
# 把NO換成0，YES換成1
df_data['Attribute22'].replace({'No':0,'Yes':1},inplace=True)
df_data['Attribute23'].replace({'No':0,'Yes':1},inplace=True)

In [87]:
# 產生訓練集和測試集
Train = df_data[ pd.notnull(df_data.Attribute23) ]
Test = df_data[ pd.isnull(df_data.Attribute23) ]

# 依照決策樹丟掉不需要的欄位
Train.drop( ['Attribute1','Attribute13','Attribute2','Attribute12','Attribute19','Attribute6','Attribute22','Attribute18','Attribute10','Attribute11','Attribute8'], axis=1, inplace=True )
Test.drop( ['Attribute1','Attribute23','Attribute13','Attribute2','Attribute12','Attribute19','Attribute6','Attribute22','Attribute18','Attribute10','Attribute11','Attribute8'], axis=1, inplace=True )

# 將測試集中的標籤欄位Attribute23單獨拆出
Y_Train = Train.Attribute23
X_Train = Train.drop( ['Attribute23'], axis=1 )

# 確認訓練集和測試集欄與列的數量
print( f'Shape of X_Train = {X_Train.shape}' )
print( f'Shape of Y_Train = {Y_Train.shape}' )
print( f'Shape of Test = {Test.shape}' )

Shape of X_Train = (6838, 11)
Shape of Y_Train = (6838,)
Shape of Test = (790, 11)


In [88]:
# 從訓練集在切0.25的資料當作測試集
train_X, test_X, train_y, test_y = train_test_split(X_Train.values, Y_Train.values, test_size=0.25,random_state=89)

# 進行標準化
my_imputer = SimpleImputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [89]:
# 載入GridSearchCV,　ShuffleSplit, XGBoost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from xgboost import XGBClassifier

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100,max_depth=4, 
                learning_rate= 0.06,
                min_child_weight=1,scale_pos_weight = 6,colsample_bytree=0.5,random_state=89)
# 使用訓練資料訓練模型
xgboostModel.fit(train_X, train_y)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(test_X)

# # 利用GridSearchCV,　ShuffleSplit調整參數、切割資料
# cv_split = ShuffleSplit(n_splits=6, train_size=0.75, test_size=0.25)
# grid_params = dict(
#     max_depth = [4, 5, 6, 7],
#     learning_rate = np.linspace(0.03, 0.3, 10),
#     n_estimators = [100, 200]
# )
# grid = GridSearchCV(xgboostModel, grid_params, cv=cv_split, scoring='neg_mean_squared_error')
# grid.fit(train_X, train_y)

In [90]:
# 顯示最佳參數
print(grid.best_params_,grid.best_score_)
print(grid.best_params_)
print('rmse:', (grid.best_score_) ** 0.5)

{'learning_rate': 0.27, 'max_depth': 7, 'n_estimators': 200} -0.14703588143525742
{'learning_rate': 0.27, 'max_depth': 7, 'n_estimators': 200}
rmse: nan


In [91]:
print('訓練集: ',xgboostModel.score(train_X,train_y))
print('測試集: ',xgboostModel.score(test_X,test_y))

訓練集:  0.8116224648985959
測試集:  0.8


In [92]:
# 進行標準化
Test_TEST_X = my_imputer.transform(Test)

In [93]:
# 把測試資料丟進去訓練好的模型內
new_predicted = xgboostModel.predict(Test_TEST_X)
new_predicted

array([0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1.,
       0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 1.

In [94]:
# 把預測結果做成csv檔
submit['ans'] = new_predicted.astype(int)
submit.to_csv( 'ans.csv', index=False )
print( f'預測結果：' )
submit 

預測結果：


,id,ans
0,0.0,0
1,1.0,1
2,2.0,0
3,3.0,0
4,4.0,0
...,...,...
785,785.0,0
786,786.0,1
787,787.0,1
788,788.0,1
